# Frontend: Gradio UI

1. This frontend provides a simple chat interface.
2. It sends user input to the FastAPI backend and displays the response.

In [ ]:
import gradio as gr
import requests

# Function to connect Gradio input to backend
def chat_fn(user_input, history):
    try:
        resp = requests.post("http://localhost:8001/chat", json={"message": user_input})
        reply = resp.json().get("response", "[Error: No response]")
    except Exception as e:
        reply = f"[Error: {e}]"
    history.append((user_input, reply))
    return history, ""

# Gradio Chatbot UI
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Type your message")
    clear = gr.Button("Clear Chat")

    def user_submit(user_input, history):
        return chat_fn(user_input, history)

    msg.submit(user_submit, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: ([], ""), None, [chatbot, msg])

demo.launch(inline=False)


# Exercise 1: Gradio UI with advanced features

In [ ]:
import gradio as gr
import requests

API_URL = "http://localhost:8001/chat"   # change if your backend runs elsewhere

# --------------------------
# Chat Function (basic)
# --------------------------
def chat_fn(user_input, history, temperature, max_tokens):
    # Basic request payload
    payload = {
        "message": user_input,
        # For now we ignore temperature/max_tokens
        # Students will later implement sending these to backend
    }
    try:
        resp = requests.post(API_URL, json=payload)
        reply = resp.json().get("response", "[Error: No response]")
    except Exception as e:
        reply = f"[Frontend error: {e}]"
    history.append((user_input, reply))
    return history, ""


# --------------------------
# Placeholder Functions (for students to implement)
# --------------------------
def save_history(history):
    # TODO: implement saving history to a file
    return "Saving not implemented yet."

def ask_about_file(file, history):
    # TODO: implement reading file and asking backend a question
    return history


# --------------------------
# UI Layout
# --------------------------
with gr.Blocks() as demo:
    with gr.Row():
        # Left: main chat area
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(label="CUI Chatbot")
            msg = gr.Textbox(label="Type your message here")
            clear = gr.Button("Clear Chat")

        # Right: sidebar with advanced features
        with gr.Column(scale=1):
            gr.Markdown("## Advanced Options")

            # Controls
            temp = gr.Slider(0, 1, value=0.7, step=0.1, label="Temperature")
            max_toks = gr.Slider(10, 500, value=200, step=10, label="Max Tokens")

            save_btn = gr.Button("💾 Save History")
            file_upload = gr.File(label="Upload a text file")


    # Chat submit
    msg.submit(chat_fn, [msg, chatbot, temp, max_toks], [chatbot, msg])
    clear.click(lambda: ([], ""), None, [chatbot, msg])

    # Advanced features (currently stubs)
    save_btn.click(save_history, [chatbot], None)
    file_upload.upload(ask_about_file, [file_upload, chatbot], [chatbot])


demo.launch(inline=False)
